In [0]:
#Trade off between DF & Temp view or DSL & SQL
#1. By default spark produces DF(mostly)
#2. DF(XLS) -> DSL functions
#3. DF(XLS) to Temp View(Temptable) -> SQL Queries
#4. DSL/SQL internally RDD transformations & actions

#We have to understand, how to represent DF to View & View to DF (natural), DF to DF (natural), View to View

In [0]:
rawdf1=spark.read.csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")#DF
print(rawdf1)
df2=rawdf1.select("*").where("age>'60'")#DSL
df2.show(2)

In [0]:
def fun1(colinput):
    return colinput.upper()

In [0]:
%python
#from pyspark.sql.functions import udf
#udffun1=udf(fun1)#works for dsl columns, but will not work for sql columns
spark.udf.register("udfsqlfun1",fun1)#works for sql columns
rawdf1.createOrReplaceTempView("rawdftv")
df2=spark.sql("select *,udfsqlfun1(firstname) from rawdftv where age>'60'")
df2.select("*").show()

In [0]:
%python
#For extraction/load operations with multiple features, preferably use DSL and not SQL
#For initial dataset creation or for ingestion/egress DSL way of creating dataframe is better, because not all the options are supported in SQL ingestion
spark.sql("""CREATE OR REPLACE TEMPORARY VIEW rawdf1view
(
  id INT,
  firstname STRING,
  lastname STRING,
  age INT,
  profession STRING
)
USING CSV
OPTIONS (
  path "/Volumes/workspace/default/volume1/custs",
  header "false",
  inferSchema "false" 
);""")

spark.sql("select * from rawdf1view limit 20").show(2)

In [0]:
original_filename='custsmodified_25/30/12.csv'
derived_datadt=original_filename.split('_')[1].split('.')[0]
spark.sql(f"""
    create or replace temp view enrichdffinal as 
    SELECT 
    custid,
    age,
    concat_ws(' ', firstname, lastname) AS fullname,
    split(concat(profession, '-', substring(profession, 1, 1)), '-')[0] AS profession,
    srcsystem AS sourcename,
    --substring(profession, 1, 1) AS profflag,
        to_date('{derived_datadt}', 'yy/dd/MM') AS datadt,
        current_date() AS loaddt,
        upper(substring(profession, 1, 3)) AS shortprof
    FROM mungeddf
""")
spark.sql("select * from enrichdffinal").show()
custid,
    age,
    fullname,
    profession,
    sourcename,
    to_date(datadt, 'yy/dd/MM') AS datadt, -- Converts string '25/30/12' to Date '2025-12-30'
    loaddt,
    shortprof


create or replace temp view enrichdf5 as 
SELECT 
    custid,
    age,
    firstname,
    lastname,
    profession,
    sourcename,
    datadt,
    loaddt
FROM enrichdf4;

create or replace temp view mergeddf as 
SELECT 
    custid,
    age,
    concat_ws(' ', firstname, lastname) AS fullname, -- Merging firstname and lastname
    split(profession, '-')[0] AS profession, -- Splitting and taking the first element
    sourcename,
    datadt,
    loaddt,
    upper(substring(split(profession, '-')[0], 1, 3)) AS shortprof -- creating shortprof based on the split result
FROM enrichdf5;

create or replace temp view formatteddf as 
SELECT 
    custid,
    age,
    fullname,
    profession,
    sourcename,
    to_date(datadt, 'yy/dd/MM') AS datadt, -- Converts string '25/30/12' to Date '2025-12-30'
    loaddt,
    shortprof
FROM mergeddf;
SELECT * FROM formatteddf LIMIT 10;